# run a model from file
This script is used to run a 10 day forecast for the whole of 2020 using saved model parameters from a file.

## load model from file

In [ ]:
# load model params:

## load obs_data:

In [ ]:
# load obs data:

In [ ]:
# get eval data from obs data:

## build functions

In [ ]:
eval_inputs, eval_targets, eval_forcings = data_utils.extract_inputs_targets_forcings(
    eval_data, target_lead_times=slice("6h", f"{eval_steps*6}h"),
    **dataclasses.asdict(task_config))

In [ ]:
# build functions + jit:

## run model


In [ ]:
# @title Autoregressive rollout (loop in python)

print("Inputs:  ", eval_inputs.dims.mapping)
print("Targets: ", eval_targets.dims.mapping)
print("Forcings:", eval_forcings.dims.mapping)

predictions = rollout.chunked_prediction(
    run_forward_jitted,
    rng=jax.random.PRNGKey(0),
    inputs=eval_inputs,
    targets_template=eval_targets * np.nan,
    forcings=eval_forcings)
predictions

In [ ]:
# loop:
N = eval_data.sizes['time'] - eval_steps - 4
# Open a zarr store to write predictions to disk
store = xr.backends.ZarrStore("../preds/predictions.zarr", "w")

for i in range(10):
    eval_batch = eval_data.isel(time=slice(i, i + eval_steps + 2))
    eval_batch = eval_batch.compute()

    eval_inputs, eval_targets, eval_forcings = data_utils.extract_inputs_targets_forcings(
    eval_batch, target_lead_times=slice("6h", f"{eval_steps*6}h"),
    **dataclasses.asdict(task_config))

    predictions = rollout.chunked_prediction(
        run_forward_jitted,
        rng=jax.random.PRNGKey(0),
        inputs=eval_inputs,
        targets_template=eval_targets * np.nan,
        forcings=eval_forcings)
    # Write the prediction dataset to the zarr store
    predictions.to_zarr(store, group=f"prediction_time_step_{prediction_time_step}", mode="a")

# Close the zarr store
store.close()

In [ ]:
# Read the predictions back into memory and concatenate them along the 'pred_times' dimension
predictions = xr.open_zarr("../preds/predictions.zarr", concat_dim="prediction_time_step")
predictions

In [ ]:
# Specify the path where you want to save the Zarr file
zarr_path = "Eval_preds/mse_64x32_2020.zarr"

# Save the dataset to the Zarr file
predictions.to_zarr(zarr_path, mode='w')